In [1]:
%%html
<style>
.output_subarea.output_text.output_stream.output_stdout > pre {
    width:max-content;
}
.p-Widget.jp-RenderedText.jp-OutputArea-output > pre {
   width:max-content;
}
</style>

In [2]:
import sys
import os
import operator

from pyspark.sql import Row
from pyspark.sql.functions import udf, when, rand, struct, col, lit, current_date, current_timestamp, monotonically_increasing_id
from pyspark.sql.types import StringType, IntegerType, StructField, StructType, DoubleType, ArrayType, FloatType

In [18]:
import visualize
from com.ophelia.OpheliaVendata import OpheliaVendata

In [5]:
module_path = os.path.abspath(os.path.join('..'))

In [6]:
ophelia = OpheliaVendata("Risk Classification Demo")
sc = ophelia.ophelia_spark.build_spark_context()
sc.addFile(module_path+"/enquire/", recursive = True)


17:27:37.602 Ophelia [INFO] ¡Hi! My name is Ophelia
17:27:37.602 Ophelia [INFO] I am an artificial assistant for machine learning applications in Spark
17:27:37.602 Ophelia [INFO] Welcome to Ophelia Assisted Intelligence System (OAIS)
17:27:37.602 Ophelia [INFO] V for Vendata...

                                 - By. Vendata-Gentleman Club -                            

                   █ █ █ █ █ █ █ █ █ █ █ █ █ █ █ █ █ █ █ █ █ █ █ █ █ █ █ █ █ █ █
                   █ █ █ █ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ █ █ █ █
                   █ █ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ █ █
                   █ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ █
                   █ ╬ ╬ ╬ █ █ █ █ █ █ █ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ █ █ █ █ █ █ █ ╬ ╬ ╬ █
                   █ ╬ ╬ █ █ ╬ ╬ ╬ ╬ █ █ █ ╬ ╬ ╬ ╬ ╬ ╬ ╬ █ █ █ ╬ ╬ ╬ ╬ █ █ ╬ ╬ █
                   █ ╬ █ █ ╬ ╬ ╬ ╬ ╬ ╬ ╬ █ █ ╬ ╬ ╬ ╬ ╬ █ █ ╬ ╬ ╬ ╬ ╬ ╬ ╬ █ █ ╬ █
                   █ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬

In [7]:
spark = ophelia.ophelia_session
path = "data/master/ophelia/data/OpheliaData/risk_classification/"
customer_data = spark.read.parquet(path)
customer_data.printSchema()
customer_data.show(5, False)

root
 |-- age: string (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- child: float (nullable = true)
 |-- saving: float (nullable = true)
 |-- insight: float (nullable = true)
 |-- backup: float (nullable = true)

+---+----------+-------+---------+------+-----+------+-------+------+
|age|job       |marital|education|gender|child|saving|insight|backup|
+---+----------+-------+---------+------+-----+------+-------+------+
|59 |admin.    |married|secondary|female|0.0  |1.0   |0.0    |1.0   |
|56 |admin.    |married|secondary|female|1.0  |0.0   |1.0    |0.0   |
|41 |technician|married|secondary|male  |1.0  |0.0   |1.0    |1.0   |
|55 |services  |married|secondary|male  |0.0  |1.0   |0.0    |0.0   |
|54 |admin.    |married|tertiary |female|0.0  |1.0   |1.0    |0.0   |
+---+----------+-------+---------+------+-----+------+-------+------+
only showing top 5 rows



In [8]:
column_selection = [
    col('age').cast('int').alias('age'),
    col('job').cast('string').alias('job'),
    col('marital').cast('string').alias('marital'),
    col('education').cast('string').alias('education'),
    col('gender').cast('string').alias('gender'),
    col('child').cast('float').alias('child'),
    col('saving').cast('float').alias('saving'),
    col('insight').cast('float').alias('insight'),
    col('backup').cast('float').alias('backup')
]

struct_assembler = customer_data.select(*column_selection, struct(column_selection).alias("struct_assembler"))
struct_assembler.show(5, False)
struct_assembler.printSchema()

+---+----------+-------+---------+------+-----+------+-------+------+--------------------------------------------------------------+
|age|job       |marital|education|gender|child|saving|insight|backup|struct_assembler                                              |
+---+----------+-------+---------+------+-----+------+-------+------+--------------------------------------------------------------+
|59 |admin.    |married|secondary|female|0.0  |1.0   |0.0    |1.0   |[59, admin., married, secondary, female, 0.0, 1.0, 0.0, 1.0]  |
|56 |admin.    |married|secondary|female|1.0  |0.0   |1.0    |0.0   |[56, admin., married, secondary, female, 1.0, 0.0, 1.0, 0.0]  |
|41 |technician|married|secondary|male  |1.0  |0.0   |1.0    |1.0   |[41, technician, married, secondary, male, 1.0, 0.0, 1.0, 1.0]|
|55 |services  |married|secondary|male  |0.0  |1.0   |0.0    |0.0   |[55, services, married, secondary, male, 0.0, 1.0, 0.0, 0.0]  |
|54 |admin.    |married|tertiary |female|0.0  |1.0   |1.0    |0.0   |

In [9]:
def build_tree_classifier():
    schema_tree = ArrayType(StructType([
        StructField("vote", FloatType(), False),
        StructField("weight", FloatType(), False),
        StructField("risk_label", StringType(), False)
    ]))
    return udf(ophelia.ophelia_classifier.tree_generator, schema_tree)

def build_risk_class():
    return udf(ophelia.ophelia_classifier.run_classification_risk, StringType())

In [10]:
tree_model_udf = build_tree_classifier()
risk_class_udf = build_risk_class()

In [24]:

dates_list = ["2020-01-02"]

dates_dict = {date: index for index, date in enumerate(dates_list)}
result = udf(lambda x: dates_dict[x], IntegerType())

In [26]:
result.__class__

function

In [11]:
tree_model_df = struct_assembler.select("*", tree_model_udf(col("struct_assembler")).alias("tree")).cache()
tree_model_df.show(5, False)
tree_model_df.printSchema()

+---+----------+-------+---------+------+-----+------+-------+------+--------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|age|job       |marital|education|gender|child|saving|insight|backup|struct_assembler                                              |tree                                                                                                                                                                     |
+---+----------+-------+---------+------+-----+------+-------+------+--------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|59 |admin.    |married|secondary|female|0.0  |1.0   |0.0    |1.0   |[59, admin., married, 

In [12]:
risk_class_df = tree_model_df.select(risk_class_udf(col("tree")).alias("risk_label"),
                                     *column_selection, current_date().alias("information_date"),
                                     current_timestamp().alias("model_date"))

risk_class_df.groupBy("risk_label").count().show(10, False)
risk_class_df.show(10, False)
risk_class_df.printSchema()

+----------+-----+
|risk_label|count|
+----------+-----+
|M         |533  |
|C         |782  |
|A         |3369 |
|MC        |2654 |
|MA        |3824 |
+----------+-----+

+----------+---+----------+--------+---------+------+-----+------+-------+------+----------------+--------------------------+
|risk_label|age|job       |marital |education|gender|child|saving|insight|backup|information_date|model_date                |
+----------+---+----------+--------+---------+------+-----+------+-------+------+----------------+--------------------------+
|MC        |59 |admin.    |married |secondary|female|0.0  |1.0   |0.0    |1.0   |2020-09-02      |2020-09-02 17:29:48.844817|
|MA        |56 |admin.    |married |secondary|female|1.0  |0.0   |1.0    |0.0   |2020-09-02      |2020-09-02 17:29:48.844817|
|MA        |41 |technician|married |secondary|male  |1.0  |0.0   |1.0    |1.0   |2020-09-02      |2020-09-02 17:29:48.844817|
|M         |55 |services  |married |secondary|male  |0.0  |1.0   |0.0   

In [16]:
columns_risk = risk_class_df.columns
when_conditions = (
    when(col("risk_label") == "A", lit(1.0)).otherwise(
        when(col("risk_label") == "MA", lit(2.0)).otherwise(
            when(col("risk_label") == "M", lit(3.0)).otherwise(
                when(col("risk_label") == "MC", lit(4.0)).otherwise(
                    when(col("risk_label") == "C", lit(5.0))))))
)
mapped_risk_label = risk_class_df.select((monotonically_increasing_id() + 100000000).alias("customer_id"), 
                                         when_conditions.alias("risk_label_id"), *columns_risk)
mapped_risk_label.show(5, False)

+-----------+-------------+----------+---+----------+-------+---------+------+-----+------+-------+------+----------------+--------------------------+
|customer_id|risk_label_id|risk_label|age|job       |marital|education|gender|child|saving|insight|backup|information_date|model_date                |
+-----------+-------------+----------+---+----------+-------+---------+------+-----+------+-------+------+----------------+--------------------------+
|100000000  |4.0          |MC        |59 |admin.    |married|secondary|female|0.0  |1.0   |0.0    |1.0   |2020-09-02      |2020-09-02 17:32:36.372511|
|100000001  |2.0          |MA        |56 |admin.    |married|secondary|female|1.0  |0.0   |1.0    |0.0   |2020-09-02      |2020-09-02 17:32:36.372511|
|100000002  |2.0          |MA        |41 |technician|married|secondary|male  |1.0  |0.0   |1.0    |1.0   |2020-09-02      |2020-09-02 17:32:36.372511|
|100000003  |3.0          |M         |55 |services  |married|secondary|male  |0.0  |1.0   |0.0

In [17]:
riskClassPath = ophelia.ophelia_write.write_parquet(
    df=mapped_risk_label,
    output_type="model",
    project="RiskClassifier",
    part="information_date"
)

17:33:14.982 Ophelia [INFO] Write Parquet
17:33:19.085 Ophelia [INFO] Write Parquet Successfully
17:33:19.085 Ophelia [INFO] New Parquet Path: data/ophelia/out/model/RiskClassifier/information_date=yyy-MM-dd
